In [1]:
import keras
import requests
from tqdm import tqdm

api_key = u'86a71105d612be1bad425258acc1c001'
api_secret = u'ec46bf1d93373fad'

RETRIEVE_IMAGES = False

IMAGE_SIZE = 128
NUM_IMAGES_PER_CLASS = 150

In [2]:
classes = ["happy", "sad", "angry", "surprised", "disgusted", "fearful", "neutral"]

In [3]:
if RETRIEVE_IMAGES:
    import flickrapi
    import os
    import requests
    flickr = flickrapi.FlickrAPI(api_key, api_secret, format='parsed-json')
    print("Retrieving images from Flickr...")
    for c in classes:
        print("Retrieving images for class {}...".format(c))
        
        try:
            os.mkdir('./images/{}'.format(c))
        except:
            pass
        
        photos = flickr.photos.search(text=c, per_page=NUM_IMAGES_PER_CLASS, sort="relevance")
        # urls = []
        with (open('./images/{}/{}.txt'.format(c, c), 'w')) as f:
            for i, photo in tqdm(enumerate(photos['photos']['photo'])):
                url = 'https://farm{}.staticflickr.com/{}/{}_{}.jpg'.format(photo['farm'], photo['server'], photo['id'], photo['secret'])
                f.write(photo['id'] + '\n')
                image = requests.get(url)
                with open('./images/{}/{}.jpg'.format(c, photo['id']), 'wb') as file:
                    file.write(image.content)

In [4]:
import numpy as np
from PIL import Image
import requests
from io import BytesIO
from sklearn.model_selection import train_test_split

In [5]:
X = np.zeros((NUM_IMAGES_PER_CLASS * len(classes), IMAGE_SIZE, IMAGE_SIZE, 3))
y = np.zeros((NUM_IMAGES_PER_CLASS * len(classes), 1))

iteration = 0
for class_idx, c in enumerate(classes):
    with open('./images/{}/{}.txt'.format(c, c), 'r') as f:
        for i, line in tqdm(enumerate(f)):
            img = Image.open('./images/{}/{}.jpg'.format(c, line.strip()))
            img = img.resize((IMAGE_SIZE, IMAGE_SIZE))
            if img.mode == 'L':
                img = np.stack((img,) * 3, axis=-1)
            img = np.asarray(img)
            X[iteration] = img
            y[iteration] = class_idx
            iteration += 1

0it [00:00, ?it/s]

299it [00:04, 72.78it/s]
300it [00:03, 80.07it/s]
211it [00:02, 73.39it/s]


ValueError: could not broadcast input array from shape (128,128,4) into shape (128,128,3)

In [ ]:
import os
os.environ['KERAS_BACKEND'] = 'tensorflow-gpu'

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2866822732795081872
xla_global_id: -1
]


In [ ]:
model = keras.models.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(IMAGE_SIZE, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(IMAGE_SIZE, (3, 3), activation='relu'),
    keras.layers.Flatten(),
    keras.layers.Dense(IMAGE_SIZE, activation='relu'),
    keras.layers.Dense(7, activation='softmax')
])

model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# train the model
model.fit(X_train, y_train, epochs=1)
score = model.evaluate(X_test, y_test)
# save the model
model.save('model.keras')

10/10 [==============================] - 13s 1s/step - loss: 2.1703 - accuracy: 0.2286


In [ ]:
pred = model.predict(X_test)

pred = np.argmax(pred, axis=1)
y_test = np.squeeze(y_test)

print(pred)

10/10 [==============================] - 11s 1s/step
[1 3 4 3 1 3 3 3 3 4 0 0 3 3 4 3 3 4 0 0 0 0 3 1 4 0 4 4 4 4 3 1 3 4 3 3 3
 1 4 1 3 4 4 4 4 4 0 3 0 6 4 4 4 4 3 3 4 4 4 3 3 0 3 0 3 1 0 4 3 4 4 3 3 4
 4 4 3 4 4 4 4 4 0 4 3 0 1 0 4 1 4 3 1 1 4 3 1 3 3 1 4 3 4 4 3 0 0 0 0 4 0
 0 3 0 0 1 0 3 5 4 3 4 4 3 3 4 4 0 0 3 3 3 0 4 3 3 4 3 3 0 4 0 0 4 1 3 0 0
 3 1 4 4 0 4 0 4 3 4 3 3 1 4 4 3 5 0 0 3 1 0 4 4 3 3 3 3 1 3 0 1 0 3 3 4 3
 4 3 4 3 3 4 3 4 4 4 1 0 3 3 3 4 0 3 0 4 0 3 0 4 0 0 4 4 0 3 3 3 4 0 4 0 0
 4 0 4 1 0 4 3 1 0 4 1 4 1 3 4 3 4 3 3 3 0 0 3 0 4 3 3 3 4 3 4 0 4 0 3 4 3
 1 0 3 3 3 0 4 3 5 4 4 3 0 4 1 4 4 4 4 3 4 0 3 0 3 4 0 4 1 0 1 3 4 0 4 4 3
 1 0 4 0 1 4 0 0 4 3 3 0 4 3 3 0 4 0 3]


In [ ]:
print(y_test)

[5. 3. 4. 5. 0. 5. 0. 5. 3. 3. 0. 1. 6. 1. 4. 0. 1. 4. 2. 4. 4. 5. 3. 3.
 0. 3. 1. 4. 4. 2. 1. 0. 3. 2. 6. 5. 2. 0. 4. 2. 2. 1. 2. 4. 5. 3. 6. 1.
 0. 6. 3. 0. 1. 2. 6. 2. 2. 5. 3. 5. 1. 2. 0. 4. 0. 6. 6. 5. 5. 0. 2. 6.
 6. 5. 1. 4. 6. 3. 4. 0. 0. 5. 1. 6. 0. 3. 6. 4. 2. 1. 5. 5. 6. 3. 4. 6.
 2. 5. 1. 3. 4. 2. 5. 4. 2. 0. 2. 2. 1. 1. 1. 4. 6. 1. 0. 3. 0. 1. 1. 2.
 6. 5. 4. 4. 6. 5. 5. 2. 6. 3. 5. 3. 0. 4. 2. 0. 4. 4. 0. 1. 3. 0. 0. 4.
 0. 1. 6. 3. 0. 3. 4. 4. 5. 5. 3. 3. 1. 1. 6. 6. 3. 5. 6. 3. 6. 1. 2. 1.
 6. 3. 6. 4. 0. 0. 6. 5. 1. 0. 0. 3. 5. 6. 0. 2. 4. 3. 6. 4. 0. 5. 0. 4.
 3. 4. 1. 0. 0. 1. 1. 2. 2. 0. 3. 2. 4. 2. 1. 2. 0. 3. 0. 5. 0. 5. 2. 1.
 0. 4. 4. 4. 1. 2. 6. 1. 5. 3. 6. 3. 0. 1. 5. 0. 1. 6. 6. 3. 4. 1. 5. 0.
 1. 6. 2. 4. 0. 5. 4. 0. 1. 5. 2. 3. 1. 1. 5. 5. 2. 5. 3. 1. 0. 5. 5. 0.
 6. 5. 1. 0. 4. 2. 3. 5. 6. 3. 4. 4. 4. 4. 3. 4. 2. 6. 4. 6. 4. 1. 4. 6.
 5. 1. 5. 2. 1. 6. 6. 3. 1. 3. 0. 2. 2. 5. 5. 4. 6. 3. 0. 4. 2. 2. 3. 2.
 4. 4. 6.]
